<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/seirs_dem_saz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
import time

# Função que simula o modelo SEIRS com beta sazonal e transmissão por frequência
def simula_seirs(beta_base, beta_amplitude, nu, mu, alpha, sigma, gamma, omega, days, S0, E0, I0, N0):
    passos = days + 1
    tempo = np.arange(0, passos)

    S = np.zeros(passos)
    E = np.zeros(passos)
    I = np.zeros(passos)
    R = np.zeros(passos)
    N = np.zeros(passos)

    # Condições iniciais
    N[0] = N0
    S[0] = S0
    E[0] = E0
    I[0] = I0
    R[0] = max(N0 - S0 - E0 - I0, 0)

    for n in range(passos - 1):
        beta_n = beta_base * (1 + beta_amplitude * np.cos(2 * np.pi * tempo[n] / 365))
        S[n+1] = S[n] + nu * N[n] - beta_n * S[n] * I[n] / N[n] - mu * S[n] + omega * R[n]
        E[n+1] = E[n] + beta_n * S[n] * I[n] / N[n] - (sigma + mu) * E[n]
        I[n+1] = I[n] + sigma * E[n] - (gamma + mu + alpha) * I[n]
        R[n+1] = R[n] + gamma * I[n] - (mu + omega) * R[n]
        N[n+1] = S[n+1] + E[n+1] + I[n+1] + R[n+1]

        S[n+1] = max(S[n+1], 0)
        E[n+1] = max(E[n+1], 0)
        I[n+1] = max(I[n+1], 0)
        R[n+1] = max(R[n+1], 0)
        N[n+1] = max(N[n+1], 0)

    return tempo, I

# Função para criar a interface interativa
def interactive_seirs():
    # Define os sliders com valores padrão realistas para gripe
    sliders = {
        "beta_base": FloatSlider(value=0.5, min=0.1, max=1.0, step=0.01, description='β base'),
        "beta_amplitude": FloatSlider(value=0.6, min=0.0, max=1.0, step=0.01, description='β amp'),
        "nu": FloatSlider(value=0.000038, min=0.0, max=0.001, step=0.00001, description='ν', readout_format='.6f'),
        "mu": FloatSlider(value=0.000022, min=0.0, max=0.001, step=0.00001, description='μ', readout_format='.6f'),
        "alpha": FloatSlider(value=0.00005, min=0.0, max=0.001, step=0.00001, description='α', readout_format='.6f'),
        "sigma": FloatSlider(value=1/2, min=0.1, max=1.0, step=0.01, description='σ', readout_format='.3f'),
        "gamma": FloatSlider(value=1/5, min=0.01, max=1.0, step=0.01, description='γ', readout_format='.3f'),
        "omega": FloatSlider(value=1/365, min=0.0, max=0.01, step=0.0001, description='ω', readout_format='.4f'),
        "days": IntSlider(value=3650, min=30, max=10000, step=1, description='Dias'),
        "S0": IntSlider(value=95000, min=0, max=100000, step=100, description='S0'),
        "E0": IntSlider(value=100, min=0, max=1000, step=10, description='E0'),
        "I0": IntSlider(value=50, min=0, max=1000, step=10, description='I0'),
        "N0": IntSlider(value=100000, min=1000, max=200000, step=1000, description='N0')
    }

    # Função para atualizar o gráfico
    def update_plot(**kwargs):
        start_time = time.process_time()
        tempo, I = simula_seirs(**kwargs)
        end_time = time.process_time()
        cpu_time = end_time - start_time
        print(f"Tempo de CPU para simulação: {cpu_time:.4f} segundos")

        tempo_anos = tempo / 365.0

        # Cria a figura com alta resolução (dpi=300)
        plt.figure(figsize=(10, 6), dpi=300)
        plt.plot(tempo_anos, I, lw=2, color='red')
        plt.title("Evolução da Gripe Sazonal - Modelo SEIRS Sazonal")
        plt.xlabel("Anos")
        plt.ylabel("Infectados (I)")
        plt.grid()

        # Salva o gráfico em alta resolução
        plt.savefig("gripe_sazonal_alta_resolucao.png", dpi=300, bbox_inches='tight')
        print("Gráfico salvo como 'gripe_sazonal_alta_resolucao.png' com DPI=300")

        plt.show()

    # Cria a interface interativa
    interact(update_plot, **sliders)

# Executa a interface interativa
interactive_seirs()

interactive(children=(FloatSlider(value=0.5, description='β base', max=1.0, min=0.1, step=0.01), FloatSlider(v…